In [14]:
!ls data/han/자동/

가평_2013.xlsx	가평_2019.xlsx	서상_2017.xlsx	  의암호_2015.xlsx
가평_2014.xlsx	가평_2020.xlsx	서상_2018.xlsx	  의암호_2016.xlsx
가평_2015.xlsx	서상_2013.xlsx	서상_2019.xlsx	  의암호_2017.xlsx
가평_2016.xlsx	서상_2014.xlsx	서상_2020.xlsx	  의암호_2018.xlsx
가평_2017.xlsx	서상_2015.xlsx	의암호_2013.xlsx  의암호_2019.xlsx
가평_2018.xlsx	서상_2016.xlsx	의암호_2014.xlsx  의암호_2020.xlsx


In [12]:
!pwd

/home/jaewon/work/water-quality


In [59]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
import datetime

In [60]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [61]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [62]:

folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'
# folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강



# ## 자동
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
# #자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [63]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,2:'

In [70]:
def make_columns(df):
    #column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    column_list = ['ymdh', 'stn_name', 'tmpr_value', 'ph_value','do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [71]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [72]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        
        print(type(date_time))
        
        
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [73]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [74]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [75]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

가평_2016.xlsx
(8784, 47)
(8784, 47)
가평_2017.xlsx
(8760, 47)
(8760, 47)
가평_2018.xlsx
(8760, 47)
(8760, 47)
가평_2019.xlsx
(8760, 47)
(8760, 47)
<class 'pandas.core.series.Series'>
의암호_2016.xlsx
(8784, 45)
(8784, 45)
의암호_2017.xlsx
(8760, 45)
(8760, 45)
의암호_2018.xlsx
(8760, 45)
(8760, 45)
의암호_2019.xlsx
(8760, 45)
(8760, 45)
<class 'pandas.core.series.Series'>
서상_2016.xlsx
(8784, 45)
(8784, 45)
서상_2017.xlsx
(8760, 45)
(8760, 45)
서상_2018.xlsx
(8760, 45)
(8760, 45)
서상_2019.xlsx
(8760, 45)
(8760, 45)
<class 'pandas.core.series.Series'>


In [76]:
train_mean, train_std

(tmpr_value    1.371507e+01
 ph_value      7.362316e+00
 do_value      1.048605e+01
 ec_value      1.062212e+02
 toc_value     1.599004e+00
 총질소_값         1.617608e+00
 총인_값          8.302030e-03
 클로로필-a_값      8.671743e+00
 Day sin       6.037103e-15
 Day cos       2.747682e-15
 Year sin     -5.681266e-08
 Year cos      2.053380e-05
 dtype: float64,
 tmpr_value     7.365772
 ph_value       0.481512
 do_value       2.071725
 ec_value      13.559850
 toc_value      0.356210
 총질소_값          0.398083
 총인_값           0.007754
 클로로필-a_값       9.530924
 Day sin        0.707110
 Day cos        0.707110
 Year sin       0.707103
 Year cos       0.707117
 dtype: float64)

## 

In [77]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [78]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [79]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [80]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [81]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [82]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [83]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [84]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [85]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [86]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [87]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [88]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [89]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [90]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    19     3     0]
 [    3     8     1    19]
 [    4    12     2    27]
 ...
 [ 5314     9     9 17840]
 [ 5323     3     3 17849]
 [ 5326     6     1 17852]]
miss_data file saved


In [91]:
dgen = GainDataGenerator(df)

pattern none


In [92]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,Year sin,Year cos
0,4.46667,7.5,10.80000,126.00000,1.35833,2.39833,0.00600,5.74167,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.30000,7.5,10.80000,126.25000,1.35000,2.48958,0.00300,5.37500,2.588190e-01,0.965926,-0.001950,0.999998
2,4.40833,7.5,10.80000,126.08333,1.38333,2.54467,0.00300,5.37500,5.000000e-01,0.866025,-0.001233,0.999999
3,4.41667,7.5,10.80000,126.00000,1.40000,2.48175,0.00225,5.44167,7.071068e-01,0.707107,-0.000516,1.000000
4,4.40000,7.5,10.80000,126.00000,1.35833,2.45333,0.00258,5.44167,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.99167,7.4,12.16167,110.81667,1.60167,1.58417,0.00900,13.66000,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.46667,7.4,12.32000,111.60833,1.64167,1.49883,0.00900,13.13583,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.12500,7.4,12.39083,112.32500,1.68167,1.58917,0.00925,12.69167,-7.071068e-01,0.707107,-0.004301,0.999991
35062,2.83333,7.4,12.45083,113.39167,1.64833,1.48750,0.01017,12.56000,-5.000000e-01,0.866025,-0.003584,0.999994


In [93]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [94]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [95]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: '용존산소'

In [96]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [97]:
val_performance = {}
performance = {}

In [98]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [99]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [100]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 0s 211ms/step - gen_loss: 125.4139 - disc_loss: 0.7249 - rmse: 1.1040 - val_loss: 1.0347
Epoch 2/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 107.0687 - disc_loss: 0.5292 - rmse: 1.0744 - val_loss: 0.8749
Epoch 3/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 81.7401 - disc_loss: 0.4699 - rmse: 0.9056 - val_loss: 0.8736
Epoch 4/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 74.5343 - disc_loss: 0.4208 - rmse: 0.8304 - val_loss: 0.7842
Epoch 5/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 61.5997 - disc_loss: 0.3881 - rmse: 0.7688 - val_loss: 0.7643
Epoch 6/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 54.0662 - disc_loss: 0.4003 - rmse: 0.7229 - val_loss: 0.7738
Epoch 7/2000
1/1 [==============================] - 0s 34ms/step - gen_loss: 48.7064 - disc_loss: 0.3911 - rmse: 0.7247 - val_loss: 0.7020
Epoc

1/1 [==============================] - 0s 31ms/step - gen_loss: 6.3522 - disc_loss: 0.3047 - rmse: 0.2961 - val_loss: 0.3189
Epoch 60/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 7.6341 - disc_loss: 0.3309 - rmse: 0.2922 - val_loss: 0.3533
Epoch 61/2000
1/1 [==============================] - 0s 33ms/step - gen_loss: 6.1784 - disc_loss: 0.3137 - rmse: 0.3416 - val_loss: 0.3212
Epoch 62/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 7.0507 - disc_loss: 0.2985 - rmse: 0.2998 - val_loss: 0.3037
Epoch 63/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 8.6037 - disc_loss: 0.3025 - rmse: 0.3189 - val_loss: 0.3437
Epoch 64/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 8.4617 - disc_loss: 0.3109 - rmse: 0.3076 - val_loss: 0.4617
Epoch 65/2000
1/1 [==============================] - 0s 33ms/step - gen_loss: 6.0761 - disc_loss: 0.2963 - rmse: 0.3050 - val_loss: 0.3704
Epoch 66/2000
1/1 [======================

Epoch 118/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 6.5747 - disc_loss: 0.2464 - rmse: 0.2981 - val_loss: 0.4508
Epoch 119/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 5.0789 - disc_loss: 0.2393 - rmse: 0.2786 - val_loss: 0.4370
Epoch 120/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 4.6082 - disc_loss: 0.2405 - rmse: 0.2719 - val_loss: 0.4381
Epoch 121/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.2613 - disc_loss: 0.2376 - rmse: 0.2630 - val_loss: 0.2716
Epoch 122/2000
1/1 [==============================] - 0s 34ms/step - gen_loss: 4.3075 - disc_loss: 0.2422 - rmse: 0.2725 - val_loss: 0.2979
Epoch 123/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 5.1514 - disc_loss: 0.2369 - rmse: 0.2888 - val_loss: 0.4429
Epoch 124/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 5.5624 - disc_loss: 0.2364 - rmse: 0.2947 - val_loss: 0.2759
Epoch 125/2000
1/1 [

1/1 [==============================] - 0s 31ms/step - gen_loss: 4.7894 - disc_loss: 0.2147 - rmse: 0.2791 - val_loss: 0.2853
Epoch 177/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 4.3219 - disc_loss: 0.2180 - rmse: 0.2701 - val_loss: 0.2882
Epoch 178/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.7939 - disc_loss: 0.2239 - rmse: 0.2774 - val_loss: 0.2832
Epoch 179/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.4135 - disc_loss: 0.2146 - rmse: 0.2716 - val_loss: 0.2660
Epoch 180/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 6.0894 - disc_loss: 0.2245 - rmse: 0.2636 - val_loss: 0.2743
Epoch 181/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 6.4773 - disc_loss: 0.2131 - rmse: 0.2624 - val_loss: 0.2855
Epoch 182/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 6.5957 - disc_loss: 0.2260 - rmse: 0.2945 - val_loss: 0.2762
Epoch 183/2000
1/1 [===============

1/1 [==============================] - 0s 31ms/step - gen_loss: 4.0359 - disc_loss: 0.2066 - rmse: 0.2585 - val_loss: 0.2907
Epoch 235/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.2493 - disc_loss: 0.1938 - rmse: 0.2465 - val_loss: 0.2630
Epoch 236/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 5.7639 - disc_loss: 0.1944 - rmse: 0.2665 - val_loss: 0.2896
Epoch 237/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 5.9463 - disc_loss: 0.1947 - rmse: 0.2650 - val_loss: 0.2597
Epoch 238/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.9441 - disc_loss: 0.2011 - rmse: 0.2538 - val_loss: 0.2697
Epoch 239/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 5.3728 - disc_loss: 0.1967 - rmse: 0.2412 - val_loss: 0.2871
Epoch 240/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.4272 - disc_loss: 0.1945 - rmse: 0.2395 - val_loss: 0.2434
Epoch 241/2000
1/1 [===============

...

1/1 [==============================] - 0s 31ms/step - gen_loss: 4.8759 - disc_loss: 0.1529 - rmse: 0.2879 - val_loss: 0.2524
Epoch 525/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 3.4106 - disc_loss: 0.1511 - rmse: 0.2588 - val_loss: 0.3065
Epoch 526/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.3304 - disc_loss: 0.1532 - rmse: 0.2845 - val_loss: 0.2825
Epoch 527/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.1591 - disc_loss: 0.1470 - rmse: 0.2507 - val_loss: 0.2468
Epoch 528/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 3.0733 - disc_loss: 0.1518 - rmse: 0.2630 - val_loss: 0.2555
Epoch 529/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.8541 - disc_loss: 0.1468 - rmse: 0.2502 - val_loss: 0.2675
Epoch 530/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 4.1039 - disc_loss: 0.1506 - rmse: 0.2460 - val_loss: 0.2799
Epoch 531/2000
1/1 [===============

1/1 [==============================] - 0s 31ms/step - gen_loss: 3.1526 - disc_loss: 0.1560 - rmse: 0.2738 - val_loss: 0.2509
Epoch 583/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 5.0094 - disc_loss: 0.1588 - rmse: 0.2811 - val_loss: 0.4131
Epoch 584/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.8166 - disc_loss: 0.1521 - rmse: 0.2380 - val_loss: 0.2331
Epoch 585/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.8423 - disc_loss: 0.1494 - rmse: 0.2323 - val_loss: 0.2618
Epoch 586/2000
1/1 [==============================] - 0s 30ms/step - gen_loss: 3.0716 - disc_loss: 0.1527 - rmse: 0.2652 - val_loss: 0.4088
Epoch 587/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.8339 - disc_loss: 0.1516 - rmse: 0.2509 - val_loss: 0.3913
Epoch 588/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.1450 - disc_loss: 0.1580 - rmse: 0.2480 - val_loss: 0.2644
Epoch 589/2000
1/1 [===============

1/1 [==============================] - 0s 32ms/step - gen_loss: 2.4745 - disc_loss: 0.1459 - rmse: 0.2266 - val_loss: 0.2501
Epoch 641/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 3.8701 - disc_loss: 0.1486 - rmse: 0.2747 - val_loss: 0.4235
Epoch 642/2000
1/1 [==============================] - 0s 214ms/step - gen_loss: 2.8123 - disc_loss: 0.1456 - rmse: 0.2294 - val_loss: 0.2577
Epoch 643/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.1386 - disc_loss: 0.1509 - rmse: 0.2371 - val_loss: 0.2271
Epoch 644/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.7617 - disc_loss: 0.1529 - rmse: 0.2749 - val_loss: 0.2402
Epoch 645/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.8730 - disc_loss: 0.1522 - rmse: 0.2337 - val_loss: 0.2641
Epoch 646/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.0354 - disc_loss: 0.1540 - rmse: 0.2486 - val_loss: 0.2247
Epoch 647/2000
1/1 [==============

1/1 [==============================] - 0s 32ms/step - gen_loss: 2.7572 - disc_loss: 0.1494 - rmse: 0.2081 - val_loss: 0.2286
Epoch 699/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7825 - disc_loss: 0.1464 - rmse: 0.2172 - val_loss: 0.2600
Epoch 700/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 3.1001 - disc_loss: 0.1485 - rmse: 0.2374 - val_loss: 0.2892
Epoch 701/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.8301 - disc_loss: 0.1469 - rmse: 0.2767 - val_loss: 0.2476
Epoch 702/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.6117 - disc_loss: 0.1499 - rmse: 0.2102 - val_loss: 0.2736
Epoch 703/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.7579 - disc_loss: 0.1434 - rmse: 0.2229 - val_loss: 0.2531
Epoch 704/2000
1/1 [==============================] - 0s 32ms/step - gen_loss: 2.7884 - disc_loss: 0.1453 - rmse: 0.2374 - val_loss: 0.3064
Epoch 705/2000
1/1 [===============

1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7082 - disc_loss: 0.1412 - rmse: 0.2466 - val_loss: 0.2490
Epoch 757/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 3.0004 - disc_loss: 0.1428 - rmse: 0.3919 - val_loss: 0.2448
Epoch 758/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7994 - disc_loss: 0.1405 - rmse: 0.2600 - val_loss: 0.2597
Epoch 759/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7327 - disc_loss: 0.1440 - rmse: 0.2427 - val_loss: 0.2281
Epoch 760/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7009 - disc_loss: 0.1405 - rmse: 0.2319 - val_loss: 0.2619
Epoch 761/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.7131 - disc_loss: 0.1433 - rmse: 0.3702 - val_loss: 0.2619
Epoch 762/2000
1/1 [==============================] - 0s 31ms/step - gen_loss: 2.9578 - disc_loss: 0.1428 - rmse: 0.3870 - val_loss: 0.2369
Epoch 763/2000
1/1 [===============

**학습 loss history 출력**

In [101]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [102]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 2s 17ms/step - loss: 0.2429


0.2429133951663971

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [106]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: '총유기탄소'

In [107]:
norm_df = pd.concat(df,axis=0)

In [108]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 12)
120
x.shape = (105120, 12)
x_reshape.shape = (876, 120, 12)
143409
y_true.shape= (105120, 12)
143409


(1, 120, 12)

In [109]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 947us/step - loss: 0.0000e+00


0.0

In [110]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [111]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 12) (120, 12)
(105192, 12)


In [112]:
y_pred[~np.isnan(data)] = np.nan

In [113]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [114]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 12)
120
n= 105120
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -1.251486  0.282415  1.457973  0.150479 -0.558340  1.960215 -0.302267   
1     -1.278638  0.282415  1.457973  0.150479 -0.558340  2.191326 -0.688669   
2     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.329489 -0.688669   
3     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.171229 -0.817470   
4     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.098379 -0.688669   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.373671 -0.546331  0.646865  1.019280  0.560945  0.252009 -0.817470   
35060 -1.387248 -0.546331  0.646865  1.019280  0.281124  0.272106 -0.946270   
35061 -1.400824 -0.546331  0.646865  1.019280  0.281124  0.302251 -0.817470   
35062 -1.400824 -0.546331  0.646865  1.019280  0.281124  0.264570 -0.817470   
35063 -1.414400 -0.546331  0.646865  1.019280  0.281124  0.219352 -0.817470   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-1.251486,0.282415,1.457973,0.150479,-0.558340,1.960215,-0.302267,-0.312016,-1.999586e-12,1.414207,...,-0.242196,-0.238574,-1.265062,0.903974,-0.016769,1.067547,-0.838161,0.299441,-0.008816,-0.616276
1,-1.278638,0.282415,1.457973,0.150479,-0.558340,2.191326,-0.688669,-0.343491,3.660237e-01,1.366019,...,0.030316,-0.144148,-1.292214,0.903974,-0.016769,1.067547,-0.838161,0.228512,0.200498,-0.626768
2,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.329489,-0.688669,-0.343491,7.071034e-01,1.224739,...,-0.043823,-0.196607,-1.332943,0.903974,0.056968,0.971013,-0.838161,0.161226,0.173730,-0.626768
3,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.171229,-0.817470,-0.343491,9.999952e-01,0.999995,...,-0.106533,-0.343491,-1.373671,0.696788,0.056968,0.971013,-0.838161,0.179683,0.094122,-0.616276
4,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.098379,-0.688669,-0.343491,1.224739e+00,0.707103,...,-0.112602,-0.188068,-1.400824,0.696788,-0.016769,0.971013,-0.838161,0.182235,0.159857,-0.637260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.391230,-0.960704,0.646865,-0.573522,0.560945,1.126209,0.084135,-0.238574,-1.366019e+00,0.366024,...,-0.322358,1.177811,0.282620,-0.546331,-1.638984,-0.380455,0.001303,-1.009049,1.758544,-0.350360
24524,0.377654,-1.167890,0.573128,-0.670055,0.560945,1.204083,-0.044666,-0.280541,-1.224739e+00,0.707103,...,-0.169787,1.167320,0.241892,-0.753517,-1.638984,-0.525255,0.001303,-1.074362,1.887344,-0.359677
24525,0.377654,-1.167890,0.573128,-0.670055,0.560945,1.214131,0.084135,-0.280541,-9.999952e-01,0.999995,...,-0.278354,1.167320,0.174011,-0.960704,-1.565247,-0.621789,0.001303,-1.091947,1.887344,-0.427409
24526,0.377654,-1.167890,0.573128,-0.670055,0.560945,1.221668,0.084135,-0.280541,-7.071034e-01,1.224739,...,-0.310980,1.156828,0.119706,-0.960704,-1.565247,-0.766589,0.001303,-1.074362,1.758544,-0.266485


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [55]:
MAX_EPOCHS = 150
target_col = '총유기탄소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

1

In [56]:
out_features

[4]

In [57]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [58]:
OUT_STEPS = 24*5

In [59]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [60]:
# train_df

In [61]:
# it = iter(wdg)

In [62]:
# x,y = next(it)
# x.shape, y.shape

In [63]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [64]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [65]:
def hour_to_day_mean(array):
    
#     print(array)
    
    time = 24
    array = tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)
#     print(array)
    
    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [66]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [67]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    o = 0
    o1 = 0
    p = 0
    nse_sum1 = 0 
    nse_sum2 = 0
    pbias_sum1 = 0
    pbias_sum2 = 0
    
    
    predictions = model(inputs)
    

    
    predictions = predictions * train_std[plot_col] + train_mean[plot_col]
    labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions)
    
#     print(labels)
    o1 = np.mean(labels.numpy())
#     print('o1')
#     print(o1)
    
    if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        o = label_temp[4].numpy()
        p = pred_temp[4].numpy()
        
#         print('pred_temp')
#         print(pred_temp[4])
#         print('+++++++++++++')
#         print(label_temp[4])
        
#         print('p')
#         print(p)
#         print('o')
#         print(o)
        
        temp_m = o-p
#         print('temp')
#         print(temp_m)
#         print(o)

#         print(o-o1)
#         print('22222')
        
        nse_sum1 = nse_sum1 + temp_m**2
        nse_sum2 = nse_sum2 + (o-o1)**2
        
    
            
        pbias_sum1 = pbias_sum1 + temp_m
        pbias_sum2 = pbias_sum2 + o
            
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
    
    
#     print(nse_sum1)
#     print('111111')
#     print(nse_sum2)

#     print('nse_sum1')
#     print(nse_sum1)
#     print('nse_sum2')
#     print(nse_sum2)
#     print('pbias_sum1')
#     print(pbias_sum1)
#     print('pbias_sum2')
#     print(pbias_sum2)
    
        
    nse = 1 - (nse_sum1 / nse_sum2)
    pbias = (pbias_sum1 / pbias_sum2) * 100
    
    
    
    
    print('NSE')
    print(nse)
    
    print('PBIAS')
    print(pbias)
    
    return nse, np.abs(pbias)
    
        
#     mae = mean_absolute_error(label_arr, pred_arr)
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
    

WindowGenerator.compa = compa

In [68]:
# np.sqrt(-0.28892756)

In [69]:
train_df.shape

(24528, 28)

In [70]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [71]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [72]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [73]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [74]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [75]:
multi_val_performance = {}
multi_performance = {}

**Base Line**

In [76]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])

100/100 [==============================] - 8s 81ms/step - loss: 0.1277 - mean_absolute_error: 0.2237
val performance = [0.1269625425338745, 0.22343488037586212]
test performance =  [0.2563895881175995, 0.2860916554927826]


**Linear**

In [77]:

multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/150
1/1 [==============================] - 0s 454ms/step - loss: 0.5858 - mean_absolute_error: 0.5261 - val_loss: 0.3829 - val_mean_absolute_error: 0.5094
Epoch 2/150
1/1 [==============================] - 0s 230ms/step - loss: 0.6825 - mean_absolute_error: 0.5503 - val_loss: 0.4333 - val_mean_absolute_error: 0.5344
Epoch 3/150
1/1 [==============================] - 0s 247ms/step - loss: 0.4863 - mean_absolute_error: 0.5056 - val_loss: 0.3770 - val_mean_absolute_error: 0.5054
Epoch 4/150
1/1 [==============================] - 0s 220ms/step - loss: 0.6136 - mean_absolute_error: 0.5611 - val_loss: 0.4339 - val_mean_absolute_error: 0.5422
Epoch 5/150
1/1 [==============================] - 0s 246ms/step - loss: 0.6330 - mean_absolute_error: 0.5573 - val_loss: 0.3737 - val_mean_absolute_error: 0.4872
Epoch 6/150
1/1 [==============================] - 0s 220ms/step - loss: 0.7902 - mean_absolute_error: 0.5785 - val_loss: 0.3399 - val_mean_absolute_error: 0.4637
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 233ms/step - loss: 0.3341 - mean_absolute_error: 0.4188 - val_loss: 0.3073 - val_mean_absolute_error: 0.4529
Epoch 52/150
1/1 [==============================] - 0s 210ms/step - loss: 0.4169 - mean_absolute_error: 0.4549 - val_loss: 0.2951 - val_mean_absolute_error: 0.4495
Epoch 53/150
1/1 [==============================] - 0s 213ms/step - loss: 0.4307 - mean_absolute_error: 0.4682 - val_loss: 0.2313 - val_mean_absolute_error: 0.3945
Epoch 54/150
1/1 [==============================] - 0s 259ms/step - loss: 0.4077 - mean_absolute_error: 0.4606 - val_loss: 0.3167 - val_mean_absolute_error: 0.4372
Epoch 55/150
1/1 [==============================] - 0s 216ms/step - loss: 0.3213 - mean_absolute_error: 0.3763 - val_loss: 0.3050 - val_mean_absolute_error: 0.4320
Epoch 56/150
1/1 [==============================] - 0s 220ms/step - loss: 0.4392 - mean_absolute_error: 0.4489 - val_loss: 0.2736 - val_mean_absolute_error: 0.4330
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 237ms/step - loss: 0.2570 - mean_absolute_error: 0.3707 - val_loss: 0.3494 - val_mean_absolute_error: 0.4655
Epoch 102/150
1/1 [==============================] - 0s 216ms/step - loss: 0.2958 - mean_absolute_error: 0.3964 - val_loss: 0.2668 - val_mean_absolute_error: 0.4178
Epoch 103/150
1/1 [==============================] - 0s 232ms/step - loss: 0.3025 - mean_absolute_error: 0.3940 - val_loss: 0.3598 - val_mean_absolute_error: 0.4887
Epoch 104/150
1/1 [==============================] - 0s 228ms/step - loss: 0.3864 - mean_absolute_error: 0.4176 - val_loss: 0.3365 - val_mean_absolute_error: 0.4595
Epoch 105/150
1/1 [==============================] - 0s 237ms/step - loss: 0.5217 - mean_absolute_error: 0.4781 - val_loss: 0.2954 - val_mean_absolute_error: 0.4390
Epoch 106/150
1/1 [==============================] - 0s 237ms/step - loss: 0.3594 - mean_absolute_error: 0.4584 - val_loss: 0.2526 - val_mean_absolute_error: 0.4007
Epoch 107/150
1/1 [=====

100/100 [==============================] - 8s 78ms/step - loss: 0.3017 - mean_absolute_error: 0.4354
val performance = [0.3016936779022217, 0.4353940784931183]
test performance =  [0.3157441318035126, 0.41172710061073303]


In [78]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

**Elman_RNN**

In [79]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [80]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [81]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 778ms/step - loss: 0.7594 - mean_absolute_error: 0.5849 - val_loss: 0.4687 - val_mean_absolute_error: 0.5501
Epoch 2/150
1/1 [==============================] - 0s 237ms/step - loss: 0.6101 - mean_absolute_error: 0.5459 - val_loss: 0.4263 - val_mean_absolute_error: 0.5232
Epoch 3/150
1/1 [==============================] - 0s 290ms/step - loss: 0.6574 - mean_absolute_error: 0.5616 - val_loss: 0.3798 - val_mean_absolute_error: 0.4938
Epoch 4/150
1/1 [==============================] - 0s 285ms/step - loss: 0.8326 - mean_absolute_error: 0.6110 - val_loss: 0.2864 - val_mean_absolute_error: 0.4376
Epoch 5/150
1/1 [==============================] - 0s 270ms/step - loss: 0.4201 - mean_absolute_error: 0.4371 - val_loss: 0.3382 - val_mean_absolute_error: 0.4753
Epoch 6/150
1/1 [==============================] - 0s 279ms/step - loss: 0.4492 - mean_absolute_error: 0.4395 - val_loss: 0.2848 - val_mean_absolute_error: 0.4291
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 297ms/step - loss: 0.2330 - mean_absolute_error: 0.3231 - val_loss: 0.1595 - val_mean_absolute_error: 0.3036
Epoch 52/150
1/1 [==============================] - 0s 245ms/step - loss: 0.1926 - mean_absolute_error: 0.2841 - val_loss: 0.1354 - val_mean_absolute_error: 0.2875
Epoch 53/150
1/1 [==============================] - 0s 265ms/step - loss: 0.1807 - mean_absolute_error: 0.2892 - val_loss: 0.1636 - val_mean_absolute_error: 0.3191
Epoch 54/150
1/1 [==============================] - 0s 264ms/step - loss: 0.2407 - mean_absolute_error: 0.2891 - val_loss: 0.1703 - val_mean_absolute_error: 0.3218
Epoch 55/150
1/1 [==============================] - 0s 281ms/step - loss: 0.2197 - mean_absolute_error: 0.3004 - val_loss: 0.1815 - val_mean_absolute_error: 0.3312
Epoch 56/150
1/1 [==============================] - 0s 258ms/step - loss: 0.2447 - mean_absolute_error: 0.2823 - val_loss: 0.1670 - val_mean_absolute_error: 0.3182
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 236ms/step - loss: 0.2018 - mean_absolute_error: 0.2727 - val_loss: 0.1656 - val_mean_absolute_error: 0.3065
Epoch 102/150
1/1 [==============================] - 0s 274ms/step - loss: 0.1884 - mean_absolute_error: 0.2979 - val_loss: 0.1663 - val_mean_absolute_error: 0.3133
Epoch 103/150
1/1 [==============================] - 0s 261ms/step - loss: 0.1760 - mean_absolute_error: 0.2668 - val_loss: 0.1529 - val_mean_absolute_error: 0.3038
Epoch 104/150
1/1 [==============================] - 0s 270ms/step - loss: 0.1768 - mean_absolute_error: 0.2686 - val_loss: 0.1936 - val_mean_absolute_error: 0.3369
Epoch 105/150
1/1 [==============================] - 0s 291ms/step - loss: 0.1576 - mean_absolute_error: 0.2673 - val_loss: 0.1841 - val_mean_absolute_error: 0.3330
Epoch 106/150
1/1 [==============================] - 0s 289ms/step - loss: 0.2899 - mean_absolute_error: 0.2862 - val_loss: 0.1849 - val_mean_absolute_error: 0.3369
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 85ms/step - loss: 0.1765 - mean_absolute_error: 0.2809
val performance = [0.1823548674583435, 0.33160173892974854]
test performance =  [0.17651960253715515, 0.28088444471359253]


In [82]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [84]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [85]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 946ms/step - loss: 0.6790 - mean_absolute_error: 0.5844 - val_loss: 0.3733 - val_mean_absolute_error: 0.4991
Epoch 2/150
1/1 [==============================] - 0s 268ms/step - loss: 0.7368 - mean_absolute_error: 0.6204 - val_loss: 0.3112 - val_mean_absolute_error: 0.4488
Epoch 3/150
1/1 [==============================] - 0s 323ms/step - loss: 0.7344 - mean_absolute_error: 0.5709 - val_loss: 0.3990 - val_mean_absolute_error: 0.4996
Epoch 4/150
1/1 [==============================] - 0s 308ms/step - loss: 0.6152 - mean_absolute_error: 0.5353 - val_loss: 0.3653 - val_mean_absolute_error: 0.4782
Epoch 5/150
1/1 [==============================] - 0s 299ms/step - loss: 0.6295 - mean_absolute_error: 0.5492 - val_loss: 0.2585 - val_mean_absolute_error: 0.4107
Epoch 6/150
1/1 [==============================] - 0s 283ms/step - loss: 0.6788 - mean_absolute_error: 0.5559 - val_loss: 0.3138 - val_mean_absolute_error: 0.4593
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 295ms/step - loss: 0.1783 - mean_absolute_error: 0.3029 - val_loss: 0.1709 - val_mean_absolute_error: 0.3162
Epoch 52/150
1/1 [==============================] - 0s 273ms/step - loss: 0.2786 - mean_absolute_error: 0.3417 - val_loss: 0.1945 - val_mean_absolute_error: 0.3434
Epoch 53/150
1/1 [==============================] - 0s 306ms/step - loss: 0.1552 - mean_absolute_error: 0.2705 - val_loss: 0.2306 - val_mean_absolute_error: 0.3721
Epoch 54/150
1/1 [==============================] - 0s 314ms/step - loss: 0.1956 - mean_absolute_error: 0.3043 - val_loss: 0.2007 - val_mean_absolute_error: 0.3493
Epoch 55/150
1/1 [==============================] - 0s 291ms/step - loss: 0.2116 - mean_absolute_error: 0.3182 - val_loss: 0.2061 - val_mean_absolute_error: 0.3489
Epoch 56/150
1/1 [==============================] - 0s 286ms/step - loss: 0.1916 - mean_absolute_error: 0.2904 - val_loss: 0.1861 - val_mean_absolute_error: 0.3307
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 291ms/step - loss: 0.2154 - mean_absolute_error: 0.2768 - val_loss: 0.1255 - val_mean_absolute_error: 0.2634
Epoch 102/150
1/1 [==============================] - 0s 292ms/step - loss: 0.2251 - mean_absolute_error: 0.2785 - val_loss: 0.1358 - val_mean_absolute_error: 0.2855
Epoch 103/150
1/1 [==============================] - 0s 305ms/step - loss: 0.1919 - mean_absolute_error: 0.2708 - val_loss: 0.1509 - val_mean_absolute_error: 0.3046
Epoch 104/150
1/1 [==============================] - 0s 268ms/step - loss: 0.3553 - mean_absolute_error: 0.2986 - val_loss: 0.1382 - val_mean_absolute_error: 0.2782
Epoch 105/150
1/1 [==============================] - 0s 287ms/step - loss: 0.1913 - mean_absolute_error: 0.2759 - val_loss: 0.1382 - val_mean_absolute_error: 0.2861
Epoch 106/150
1/1 [==============================] - 0s 324ms/step - loss: 0.2711 - mean_absolute_error: 0.3097 - val_loss: 0.1506 - val_mean_absolute_error: 0.2944
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 91ms/step - loss: 0.1786 - mean_absolute_error: 0.2763
val performance = [0.14730539917945862, 0.2967419922351837]
test performance =  [0.17861534655094147, 0.2763054072856903]


In [86]:
multi_window.plot2(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**LSTM**

In [87]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [88]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 0.6516 - mean_absolute_error: 0.5544 - val_loss: 0.3900 - val_mean_absolute_error: 0.5116
Epoch 2/150
1/1 [==============================] - 0s 228ms/step - loss: 0.7173 - mean_absolute_error: 0.5566 - val_loss: 0.3301 - val_mean_absolute_error: 0.4475
Epoch 3/150
1/1 [==============================] - 0s 223ms/step - loss: 0.5792 - mean_absolute_error: 0.4972 - val_loss: 0.4609 - val_mean_absolute_error: 0.5341
Epoch 4/150
1/1 [==============================] - 0s 239ms/step - loss: 0.7548 - mean_absolute_error: 0.5935 - val_loss: 0.3874 - val_mean_absolute_error: 0.5056
Epoch 5/150
1/1 [==============================] - 0s 262ms/step - loss: 0.6572 - mean_absolute_error: 0.5585 - val_loss: 0.3656 - val_mean_absolute_error: 0.4847
Epoch 6/150
1/1 [==============================] - 0s 264ms/step - loss: 0.7560 - mean_absolute_error: 0.6024 - val_loss: 0.3634 - val_mean_absolute_error: 0.4781
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 254ms/step - loss: 0.3047 - mean_absolute_error: 0.3274 - val_loss: 0.2425 - val_mean_absolute_error: 0.3860
Epoch 52/150
1/1 [==============================] - 0s 264ms/step - loss: 0.2348 - mean_absolute_error: 0.3126 - val_loss: 0.2571 - val_mean_absolute_error: 0.4053
Epoch 53/150
1/1 [==============================] - 0s 279ms/step - loss: 0.2111 - mean_absolute_error: 0.3045 - val_loss: 0.2210 - val_mean_absolute_error: 0.3558
Epoch 54/150
1/1 [==============================] - 0s 243ms/step - loss: 0.1738 - mean_absolute_error: 0.2867 - val_loss: 0.2731 - val_mean_absolute_error: 0.4090
Epoch 55/150
1/1 [==============================] - 0s 259ms/step - loss: 0.2310 - mean_absolute_error: 0.2750 - val_loss: 0.2539 - val_mean_absolute_error: 0.4020
Epoch 56/150
1/1 [==============================] - 0s 243ms/step - loss: 0.1416 - mean_absolute_error: 0.2705 - val_loss: 0.2395 - val_mean_absolute_error: 0.3930
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 252ms/step - loss: 0.1401 - mean_absolute_error: 0.2503 - val_loss: 0.2883 - val_mean_absolute_error: 0.4164
Epoch 102/150
1/1 [==============================] - 0s 269ms/step - loss: 0.1715 - mean_absolute_error: 0.2778 - val_loss: 0.2436 - val_mean_absolute_error: 0.3974
Epoch 103/150
1/1 [==============================] - 0s 221ms/step - loss: 0.1567 - mean_absolute_error: 0.2476 - val_loss: 0.2538 - val_mean_absolute_error: 0.3882
Epoch 104/150
1/1 [==============================] - 0s 239ms/step - loss: 0.1976 - mean_absolute_error: 0.2608 - val_loss: 0.3246 - val_mean_absolute_error: 0.4376
Epoch 105/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1220 - mean_absolute_error: 0.2376 - val_loss: 0.2439 - val_mean_absolute_error: 0.3717
Epoch 106/150
1/1 [==============================] - 0s 253ms/step - loss: 0.1974 - mean_absolute_error: 0.2645 - val_loss: 0.2491 - val_mean_absolute_error: 0.3718
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 114ms/step - loss: 0.3505 - mean_absolute_error: 0.4657
val performance = [0.35053834319114685, 0.46571552753448486]
test performance =  [0.1820107102394104, 0.29349932074546814]


**CNN model**

In [89]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [90]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 761ms/step - loss: 0.8272 - mean_absolute_error: 0.6390 - val_loss: 0.3776 - val_mean_absolute_error: 0.4947
Epoch 2/150
1/1 [==============================] - 0s 266ms/step - loss: 0.8282 - mean_absolute_error: 0.6244 - val_loss: 0.3988 - val_mean_absolute_error: 0.4914
Epoch 3/150
1/1 [==============================] - 0s 250ms/step - loss: 0.5112 - mean_absolute_error: 0.5034 - val_loss: 0.3190 - val_mean_absolute_error: 0.4454
Epoch 4/150
1/1 [==============================] - 0s 243ms/step - loss: 0.4846 - mean_absolute_error: 0.4673 - val_loss: 0.3455 - val_mean_absolute_error: 0.4640
Epoch 5/150
1/1 [==============================] - 0s 232ms/step - loss: 0.4181 - mean_absolute_error: 0.4865 - val_loss: 0.3015 - val_mean_absolute_error: 0.4299
Epoch 6/150
1/1 [==============================] - 0s 244ms/step - loss: 0.4366 - mean_absolute_error: 0.4810 - val_loss: 0.3347 - val_mean_absolute_error: 0.4525
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 260ms/step - loss: 0.1728 - mean_absolute_error: 0.2711 - val_loss: 0.1510 - val_mean_absolute_error: 0.3028
Epoch 52/150
1/1 [==============================] - 0s 207ms/step - loss: 0.2721 - mean_absolute_error: 0.2840 - val_loss: 0.1279 - val_mean_absolute_error: 0.2779
Epoch 53/150
1/1 [==============================] - 0s 245ms/step - loss: 0.2146 - mean_absolute_error: 0.2902 - val_loss: 0.1359 - val_mean_absolute_error: 0.2901
Epoch 54/150
1/1 [==============================] - 0s 258ms/step - loss: 0.2146 - mean_absolute_error: 0.2787 - val_loss: 0.1657 - val_mean_absolute_error: 0.3126
Epoch 55/150
1/1 [==============================] - 0s 234ms/step - loss: 0.1642 - mean_absolute_error: 0.2604 - val_loss: 0.1607 - val_mean_absolute_error: 0.3206
Epoch 56/150
1/1 [==============================] - 0s 215ms/step - loss: 0.2173 - mean_absolute_error: 0.2924 - val_loss: 0.1853 - val_mean_absolute_error: 0.3351
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 215ms/step - loss: 0.1808 - mean_absolute_error: 0.2675 - val_loss: 0.2000 - val_mean_absolute_error: 0.3450
Epoch 102/150
1/1 [==============================] - 0s 246ms/step - loss: 0.2476 - mean_absolute_error: 0.2654 - val_loss: 0.2070 - val_mean_absolute_error: 0.3467
Epoch 103/150
1/1 [==============================] - 0s 236ms/step - loss: 0.1446 - mean_absolute_error: 0.2595 - val_loss: 0.1849 - val_mean_absolute_error: 0.3307
Epoch 104/150
1/1 [==============================] - 0s 242ms/step - loss: 0.1225 - mean_absolute_error: 0.2523 - val_loss: 0.1963 - val_mean_absolute_error: 0.3318
Epoch 105/150
1/1 [==============================] - 0s 213ms/step - loss: 0.1608 - mean_absolute_error: 0.2550 - val_loss: 0.2193 - val_mean_absolute_error: 0.3522
Epoch 106/150
1/1 [==============================] - 0s 229ms/step - loss: 0.1714 - mean_absolute_error: 0.2682 - val_loss: 0.2106 - val_mean_absolute_error: 0.3483
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 118ms/step - loss: 0.2437 - mean_absolute_error: 0.3863
val performance = [0.24368496239185333, 0.3862520754337311]
test performance =  [0.20299610495567322, 0.29608064889907837]


In [91]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 107ms/step - loss: 0.2531 - mean_absolute_error: 0.3956
val performance = [0.2530602514743805, 0.39564603567123413]
test performance =  [0.17604036629199982, 0.2775328457355499]


**performance**

In [92]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
val_nse = {}
val_pbias = {}

In [94]:
val_nse['BaseLine'], val_pbias['BaseLine'] = multi_window.compa(last_baseline, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.4772043138309977
PBIAS
2.3143582884109604


In [95]:
val_nse['Linear'], val_pbias['Linear'] = multi_window.compa(multi_linear_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.40312584860888667
PBIAS
0.274612281926783


In [120]:
val_nse['Elman'], val_pbias['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.5640673150067443
PBIAS
-0.8474225570429367


In [114]:
val_nse['GRU'], val_pbias['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.7584873891869468
PBIAS
-1.1888146261093089


In [110]:
val_nse['LSTM'], val_pbias['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.6742719054527359
PBIAS
-1.8338150634618147


In [99]:
val_nse['CNN'], val_pbias['CNN'] = multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총유기탄소
NSE
0.6291582787586865
PBIAS
-3.584394612171048


In [121]:
x = np.arange(len(val_nse))
x
width = 0.35

plt.figure()
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=45)
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …